In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#Set path
path = 'drive/MyDrive/SB_practical_notebooks/data/'
#path = 'drive/MyDrive/data/'

In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 5.3 MB/s 


In [ ]:
'''
Basic parsing and iteration of Structure objects implemented by the BIO module of BioPython
Save selected residues into a new PDB file
Generate distance matrix

Bio.PDB module FAQ
https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ
'''

from Bio.PDB import PDBList, is_aa, PDBIO
from Bio.PDB.PDBParser import PDBParser
from Bio.SeqUtils import IUPACData
from Bio.PDB.PDBIO import Select
from Bio.SeqIO.PdbIO import PdbSeqresIterator


In [ ]:

# Input
pdb_id = '1cu4'
#pdb_id = '1byi'
#pdb_id = '3k8y'
#pdb_id = '1nww'


In [ ]:
# Fetch a PDB file from the website to the current dir  - How to download structures from the PDB?
pdbl = PDBList()
pdbl.retrieve_pdb_file(pdb_id, pdir=path, file_format='pdb')  # Will save to pdbXXXX.en

Structure exists: 'drive/MyDrive/SB_practical_notebooks/data/pdb1cu4.ent' 


'drive/MyDrive/SB_practical_notebooks/data/pdb1cu4.ent'

In [ ]:
# Get the SEQRES by using SeqIO
#it identifies all SEQRES in the PDB and iterates for each chain (in caso of multi-chain structures)
with open(path + "pdb{}.ent".format(pdb_id)) as f:
 seq_records = (PdbSeqresIterator(f))
 for seq_record in seq_records:
  print(seq_record)


ID: 2HHB:A
Name: 2HHB:A
Description: UNP:P01922 HBA_HUMAN
Database cross-references: UNP:P01922, UNP:HBA_HUMAN
Number of features: 0
/chain=A
/molecule_type=protein
Seq('VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQ...KYR')
ID: 2HHB:B
Name: 2HHB:B
Description: UNP:P02023 HBB_HUMAN
Database cross-references: UNP:P02023, UNP:HBB_HUMAN
Number of features: 0
/chain=B
/molecule_type=protein
Seq('VHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAV...KYH')
ID: 2HHB:C
Name: 2HHB:C
Description: UNP:P01922 HBA_HUMAN
Database cross-references: UNP:P01922, UNP:HBA_HUMAN
Number of features: 0
/chain=C
/molecule_type=protein
Seq('VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQ...KYR')
ID: 2HHB:D
Name: 2HHB:D
Description: UNP:P02023 HBB_HUMAN
Database cross-references: UNP:P02023, UNP:HBB_HUMAN
Number of features: 0
/chain=D
/molecule_type=protein
Seq('VHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAV...KYH')


In [ ]:
# Load the structure
structure = PDBParser(QUIET=True).get_structure(pdb_id, path + "pdb{}.ent".format(pdb_id))

# Iterate structure

#is_aa function check whether the residue is an amino acid or not (modifies residues, HETATM, ligands, Ions, etc)
#IUPAC Data is a package to conver from 3 residue names to one letter residue name
for model in structure:
    for chain in model:
        for residue in chain:
            if not is_aa(residue):  
              
              # Filter hetero groups (returns only amino acids)
                #residue.id tuples (unique IDs) that contains hetero_flag, position (residue index), insertion_code 

                print("model {} chain {} residue_id {} resname {} resname_3to1 {}".format(model.id, chain.id, residue.id, residue.get_resname(),
                                                        IUPACData.protein_letters_3to1.get(residue.get_resname().capitalize())))
#                 for atom in residue:
 #                    print("atom {} {} {}".format(atom.id, atom.get_bfactor(), atom.get_coord()))
                #pass
#            else:
#                print(residue.id)


model 0 chain L residue_id ('W', 215, ' ') resname HOH resname_3to1 None
model 0 chain L residue_id ('W', 216, ' ') resname HOH resname_3to1 None
model 0 chain L residue_id ('W', 217, ' ') resname HOH resname_3to1 None
model 0 chain H residue_id ('W', 214, ' ') resname HOH resname_3to1 None
model 0 chain H residue_id ('W', 215, ' ') resname HOH resname_3to1 None
model 0 chain H residue_id ('W', 216, ' ') resname HOH resname_3to1 None
model 0 chain H residue_id ('W', 217, ' ') resname HOH resname_3to1 None
model 0 chain H residue_id ('W', 218, ' ') resname HOH resname_3to1 None
model 0 chain P residue_id ('W', 2, ' ') resname HOH resname_3to1 None


In [ ]:
# Extract a list of residues between start and end positions excluding hetero and water atoms
# It assumes there are not insertion codes and residues numbers are not necessarily continuous
# Utility: domain / key regions identification and extraction

domain_residues = []
start_flag = False
domain_start = (" ", 10, " ")
domain_end = (" ", 100, " ")
for residue in structure[0]['L'].get_residues():  # Model 0, chain L
    if residue.id[0] == " ":  
      # Exclude hetero and water atoms
        # print(residue.id)
        # Get starting position, a piori I don't know where is the first residue
        if residue.id == domain_start:
            start_flag = True

        if start_flag:
            domain_residues.append(residue)
            #print(residue.id)

        # Get ending position
        if residue.id == domain_end:
            break

print(domain_residues)

[<Residue SER het=  resseq=10 icode= >, <Residue LEU het=  resseq=11 icode= >, <Residue SER het=  resseq=12 icode= >, <Residue VAL het=  resseq=13 icode= >, <Residue THR het=  resseq=14 icode= >, <Residue ILE het=  resseq=15 icode= >, <Residue GLY het=  resseq=16 icode= >, <Residue GLN het=  resseq=17 icode= >, <Residue PRO het=  resseq=18 icode= >, <Residue ALA het=  resseq=19 icode= >, <Residue SER het=  resseq=20 icode= >, <Residue ILE het=  resseq=21 icode= >, <Residue SER het=  resseq=22 icode= >, <Residue CYS het=  resseq=23 icode= >, <Residue LYS het=  resseq=24 icode= >, <Residue SER het=  resseq=25 icode= >, <Residue SER het=  resseq=26 icode= >, <Residue GLN het=  resseq=27 icode= >, <Residue SER het=  resseq=27 icode=A>, <Residue LEU het=  resseq=27 icode=B>, <Residue LEU het=  resseq=27 icode=C>, <Residue ASP het=  resseq=27 icode=D>, <Residue SER het=  resseq=27 icode=E>, <Residue ASP het=  resseq=28 icode= >, <Residue GLY het=  resseq=29 icode= >, <Residue LYS het=  resse

In [ ]:
# Save a PDB chain
class Select(Select):
    def __init__(self, chain_ids=None, residues=None):
        self.chain_ids = chain_ids
        self.residues = residues

    def accept_chain(self, chain):
        return self.chain_ids is None or chain.id in self.chain_ids

    def accept_residue(self, residue):
        return self.residues is None or residue in self.residues

    def accept_atom(self, atom):
        return not atom.is_disordered() or atom.get_altloc() == "A"



In [ ]:
# Save a PDB file containing only a list of selected residues
io = PDBIO()
io.set_structure(structure[0])
io.save(path + "pdb{}_cut.ent".format(pdb_id), select=Select(residues=domain_residues))


# Exercises
## Fetch 1CU4
## Fetch 2KKW

1.   How many chains?
2.   Total number of hetero atoms?
3. Total number of water molecules?
4. Which is the index of the last residue of chain H?
5. Total number of residues?
6. Why the total number of residues is different from the last index?
Split PDB 2KKW into different files, one file per model. (Need to write a new "Select" class for picking models)

In [ ]:
pdb_id = '1cu4'
pdbl = PDBList()
pdbl.retrieve_pdb_file(pdb_id, pdir=path, file_format='pdb')  # Will save to pdbXXXX.ent
structure = PDBParser(QUIET=True).get_structure(pdb_id, path + "pdb{}.ent".format(pdb_id))

Structure exists: 'drive/MyDrive/SB_practical_notebooks/data/pdb1cu4.ent' 


In [ ]:
#Exercise 1
print("chains", len(structure[0]))

chains 3


In [ ]:
#Exercise 2
print("hetero atoms", sum([1 for chain in structure[0] for residue in chain if residue.id[0] != ' ' for atom in residue]))

hetero atoms 9


In [ ]:
#Exercise 3
print("water molecules", sum([1 for chain in structure[0] for residue in chain if residue.id[0] == 'W']))

water molecules 9


In [ ]:
#Exercise 4
print("last residue of chain H", [residue.id for residue in structure[0]['H'] if residue.id[0] == ' '][-1])


last residue of chain H (' ', 213, ' ')


In [ ]:
#Exercise 5
print("total elements in chain H", len(structure[0]['H']))
print("total residues in chain H", len([residue for residue in structure[0]['H'] if is_aa(residue)]))

total elements in chain H 220
total residues in chain H 215


In [ ]:
#Exercise 6
print("some elements are not standard residues", [residue.id for residue in structure[0]['H'] if not is_aa(residue)])
print("some elements are water molecules", [residue.id for residue in structure[0]['H'] if residue.id[0] != ' '])
print("some residues have insertion codes", [residue.id for residue in structure[0]['H'] if residue.id[2] != ' '])

some elements are not standard residues [('W', 214, ' '), ('W', 215, ' '), ('W', 216, ' '), ('W', 217, ' '), ('W', 218, ' ')]
some elements are water molecules [('W', 214, ' '), ('W', 215, ' '), ('W', 216, ' '), ('W', 217, ' '), ('W', 218, ' ')]
some residues have insertion codes [(' ', 52, 'A'), (' ', 82, 'A'), (' ', 82, 'B'), (' ', 82, 'C')]


In [ ]:
# Define the model select class
class ModelSelect(Select):
    """
    Select model for BioPython PDB save
    """
    def __init__(self, model_ids):
        self.model_ids = model_ids

    def accept_model(self, model):
        return (model.serial_num) in self.model_ids


# Fetch 2KKW
pdb_id = '2kkw'
pdbl = PDBList()
pdbl.retrieve_pdb_file(pdb_id, pdir=path, file_format='pdb')  # Will save to pdbXXXX.ent
structure = PDBParser(QUIET=True).get_structure(pdb_id, path + "pdb{}.ent".format(pdb_id))

# Save models
io = PDBIO()
io.set_structure(structure)
for model_index, model in enumerate(structure):
    io.save(path + "pdb{}_{}.ent".format(pdb_id, model_index + 1), select=ModelSelect([model_index + 1]))
